In [1]:
import sys
from pathlib import Path
from os import getcwd

sys.path.append(str(Path(getcwd()).parent))

In [3]:
import codecs
import pandas as pd
import numpy as np
from bulletin.services.metabase import Metabase

mb = Metabase()

pd.set_option('display.max_columns', None)

enter with cookie metabase.SESSION=8d43fc6e-3286-4d0e-9e56-a2d95ba5aece



saved_queries: ['diario', 'old_diario', 'all_notifica']


In [4]:
notificacao, information_schema, constraint = mb.download_tables()

Success code 200
Saving in /home/vinnylg/projects/covid19datascience/bulletin/resources/tables/notificacao.csv
Download finish, time elapsed: 0:00:00.255048
downloaded shape (179, 3)

Success code 200
Saving in /home/vinnylg/projects/covid19datascience/bulletin/resources/tables/columns.csv
Download finish, time elapsed: 0:00:00.218354
downloaded shape (3933, 3)

Success code 200
Saving in /home/vinnylg/projects/covid19datascience/bulletin/resources/tables/notifica_constraint.csv
Download finish, time elapsed: 0:00:01.143340
downloaded shape (219, 5)

Download aux_tables time elepsed:00:00:02


In [5]:
notificacao.loc[notificacao['foreign_table_name']=='sim_nao', 'udt_name'] = 'bool'
notificacao.loc[notificacao['foreign_table_name']=='sim_nao', ['constraint_name', 'foreign_table_name', 'foreign_column_name']] = ''
notificacao

,table_name,column_name,udt_name,constraint_name,foreign_table_name,foreign_column_name
0,notificacao,id,int4,,,
1,notificacao,data_notificacao,timestamp,,,
2,notificacao,possui_cpf,bool,,,
3,notificacao,tipo_paciente,int4,tipo_paciente,tipo_paciente,id
4,notificacao,pais_residencia,varchar,pais_residencia,pais,co_pais
...,...,...,...,...,...,...
174,notificacao,lote_covid19_1a,varchar,,,
175,notificacao,data_covid19_2a,date,,,
176,notificacao,lote_covid19_2a,varchar,,,
177,notificacao,vacina_influenza,int4,,,


In [6]:
constraint = constraint.loc[constraint['foreign_table_name']!='sim_nao'] 
constraint

,constraint_name,table_name,column_name,foreign_table_name,foreign_column_name
0,ibge_residencia,notificacao,ibge_residencia,municipio,co_municipio
43,evolucao_monitoramento,monitoramento,evolucao,evolucao_monitoramento,id
44,id_contato,monitoramento,id_contato,contato,id
45,status_monitoramento,monitoramento,status,status_monitoramento,id
46,ocupacao_monitoramento,monitoramento,ocupacao,ocupacao,id
...,...,...,...,...,...
214,etnia,notificacao,etnia,etnia,co_etnia
215,status,notificacao,status_notificacao,status,id
216,paciente_institucionalizado,notificacao,paciente_institucionalizado,paciente_institucionalizado,id
217,cbo,notificacao,cod_cbo,cbo,codigo


In [7]:
with codecs.open('tables.dbml',"w","utf-8-sig") as out:

    for table_name, table in information_schema.groupby('table_name'):
        if any(table_name in n for n in notificacao['foreign_table_name'].values):
            out.write(f"Table {table_name}" + " {\n")
            for column_name, udt_name in zip(table['column_name'],table['udt_name']):
                if column_name in constraint.loc[constraint['table_name'] == table_name, 'foreign_column_name'].values:
                    out.write(f"\t {column_name} {udt_name} [pk]\n")
                    notificacao.loc[notificacao['foreign_table_name']==table_name,'ref'] = f"[ref: - {table_name}.{column_name}]"
                else:
                    out.write(f"\t {column_name} {udt_name}\n")
            out.write("}\n\n")


    out.write(f"Table notificacao" + " {\n")
    for column_name, udt_name,ref in zip(notificacao['column_name'],notificacao['udt_name'],notificacao['ref']):
        out.write(f"\t {column_name} {udt_name} {ref if ref != None else ''}\n")
    out.write("}\n\n")